In [1]:
# Inicio, importar librerias necesarias:
# pandas: tratamiento de DataFrames
# numpy: control numérido
# matplotlib: visualizaciones
# seaborn: visualizaciones avanzadas


import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
import warnings
warnings.filterwarnings("ignore")

import scipy.stats as st
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest

pd.set_option('display.max_columns', None)
madrid = pd.read_excel('Madrid.xlsx')
cyl = pd.read_excel('CYL.xlsx')
andalucia = pd.read_excel('Andalucia.xlsx')
aragon = pd.read_excel('Aragon.xlsx')
asturias = pd.read_excel('Asturias.xlsx')
baleares = pd.read_excel('Baleares.xlsx')
cym = pd.read_excel('CYMancha.xlsx')
canarias = pd.read_excel('Canarias.xlsx')
cantabria = pd.read_excel('Cantabria.xlsx')
cata = pd.read_excel('Cataluña.xlsx')
ceuta = pd.read_excel('Ceuta.xlsx')
valen = pd.read_excel('ComunidadValenciana.xlsx')
euskadi = pd.read_excel('Euskadi.xlsx')
extrema = pd.read_excel('Extremadura.xlsx')
galicia = pd.read_excel('Galicia.xlsx')
navarra = pd.read_excel('Navarra.xlsx')
melilla = pd.read_excel('Melilla.xlsx')
murcia = pd.read_excel('Murcia.xlsx')




In [21]:
spain = pd.concat([madrid,cyl,andalucia,aragon,asturias,baleares,cym,canarias,cantabria,cata,ceuta,valen,euskadi,extrema,galicia,navarra,melilla,murcia])

In [22]:
def EDA(df):
        print(f"La forma:")
        print(f"{df.shape}\n")
        print(f"\n-----------------------------\n")
        print(f"Las columnas:")
        print(f"{df.columns}\n")
        print(f"\n-----------------------------\n")
        print(f"Los tipos de datos:")
        print(f"{df.dtypes}\n")
        print(f"\n-----------------------------\n")
        print(f"Los nulos:")
        print(f"{df.isnull().sum()}\n")
        print(f"\n-----------------------------\n")
        print(f"Los duplicados:")
        print(f"{df.duplicated().sum()}\n")
        print(f"\n-----------------------------\n")
        print(f"Los principales estadísticos:")
        print(f"{df.describe().T}\n")

        print(f"Las modas de las columnas categóricas:\n")
        columnas_cat = df.select_dtypes(include = 'object')
        for columna in columnas_cat:
            if df[columna].isnull().any():
                print(f"Revisando {columna}")
                print(df[columna].value_counts())  #mode()[0]
                print("") 
        print(f"\n-----------------------------\n")
        return


def null_check(df):
    
        print(" COLUMNAS CATEGÓRICAS")
        col_obj = df.select_dtypes(include='object').columns
        #Lista para guardar columnas

        found_obj = False  #Manera de comprobar que vamos a encontra nulos las columnas categoricas
        #Recorremos el bucle columna por columna
        for col in col_obj:
            
            por = df[col].isnull().mean() * 100
            if por > 0:
                print(f" - {col}: {por:.0f}% de nulos")
                #Si el porcentaje es de X se podria comparar aqui y tratar


                found_obj = True #Check de que ha encontrado nulos

        if not found_obj:
            print(" No hay nulos en columnas categóricas")


        #Ahora comprobación de columas de tipo numérico
        print(" COLUMNAS NUMÉRICAS")
        col_num = df.select_dtypes(include='number').columns
        
        found_num = False
        for col in col_num:
            por = df[col].isnull().mean() * 100
            if por > 0:
                print(f" - {col}: {por:.0f}% de nulos")
                found_num = True
        if not found_num:
            print(" No hay nulos en columnas numéricas")


def tratar_null(df):
    
    col_obj = df.select_dtypes(include='object').columns        
    col_num = df.select_dtypes(include='number').columns
    #Lista para guardar columnas, trataremos una a una las nulas, dependiendo el tipo de dato y % de datos
    #Recorremos el bucle columna por columna
    for col in col_obj:
        por = df[col].isnull().mean() * 100
        if por == 0:
            print(f"No hay nulos en: {col} ")
        else:    
            if por < 20:
                # Pocos nulos: imputar con la moda (valor más frecuente)
                print(f"Tratando columna: {col} ({por:.0f}% nulos) -> Moda")
                df[col] = df[col].fillna(df[col].mode()[0])

            elif 20 <= por < 80:
                # Bastantes nulos: imputar con valor genérico
                print(f"Tratando columna: {col} ({por:.0f}% nulos) -> 'Unknown'")
                df[col] = df[col].fillna('Unknown')

            else:
                # Demasiados nulos: se puede optar por eliminar o marcar aparte
                print(f"Columna {col} tiene {por:.0f}% nulos. Considerar eliminarla o analizarla aparte.")
                df.drop(df[col], axis=1, inplace=True)
                #df[col] = df.drop(df[col], axis=1, inplace=True)
                

            #Si el porcentaje es de X se podria comparar aqui y tratar
            #Ahora comprobación de columas de tipo numérico
    for col in col_num:
        por = df[col].isnull().mean() * 100
        if por == 0:
                print(f"No hay nulos en: {col} ")
        
        else:    
            if por < 20:
                # Pocos nulos: imputar con la media
                print(f"Tratando columna: {col} ({por:.0f}% nulos) -> Media")
                imputer = SimpleImputer(strategy='mean')
                df[col] = imputer.fit_transform(df[[col]])

            elif 21 <= por < 70:
                # Porcentaje medio-alto de nulos: imputar con KNN
                print(f"Tratando columna: {col} ({por:.0f}% nulos) -> KNNImputer")
                imputer_knn = KNNImputer(n_neighbors=5)
                df[col] = imputer_knn.fit_transform(df[[col]])

                # Cambio

            elif por > 71:
            # Demasiados nulos: imputar con un valor neutro o eliminar
                print(f"Columna {col} tiene {por:.0f}% nulos. Considerar eliminarla o imputar manualmente.")
                #df=df.drop(df[col], axis=1)
        #Ya tratadito devolvemos el df
    
    return df
                       


In [9]:
trimestres_dict = {
    1: "T1", 2: "T1", 3: "T1",
    4: "T2", 5: "T2", 6: "T2",
    7: "T3", 8: "T3", 9: "T3",
    10: "T4", 11: "T4", 12: "T4"
}


In [24]:
spain['trimestre'] = spain['Mes'].map(trimestres_dict)


In [ ]:
spain['Mes'] = spain['Mes'].astype('Int64')

In [26]:
EDA(spain)

La forma:
(4124, 7)


-----------------------------

Las columnas:
Index(['Fecha', 'Precio m2', 'Provincia', 'Comunidad', 'Mes', 'Año',
       'trimestre'],
      dtype='object')


-----------------------------

Los tipos de datos:
Fecha        datetime64[ns]
Precio m2            object
Provincia            object
Comunidad            object
Mes                 float64
Año                   int64
trimestre            object
dtype: object


-----------------------------

Los nulos:
Fecha        0
Precio m2    0
Provincia    0
Comunidad    0
Mes          0
Año          0
trimestre    4
dtype: int64


-----------------------------

Los duplicados:
0


-----------------------------

Los principales estadísticos:
        count                           mean                  min  \
Fecha    4124  2022-05-02 23:24:23.045586944  2018-02-01 00:00:00   
Mes    4124.0                       6.346117                  1.0   
Año    4124.0                    2021.891125               2018.0   

     

In [23]:
tratar_null(spain)

No hay nulos en: Precio m2 
No hay nulos en: Provincia 
Tratando columna: Comunidad (0% nulos) -> Moda
Tratando columna: Mes (0% nulos) -> Media
No hay nulos en: Año 


,Fecha,Precio m2,Provincia,Comunidad,Mes,Año
0,2025-09-01,5.950 €/m2,Madrid,Madrid,9.0,2025
1,2025-08-01,5.867 €/m2,Madrid,Madrid,8.0,2025
2,2025-07-01,5.750 €/m2,Madrid,Madrid,7.0,2025
3,2025-06-01,5.729 €/m2,Madrid,Madrid,6.0,2025
4,2025-05-01,5.669 €/m2,Madrid,Madrid,5.0,2025
...,...,...,...,...,...,...
76,2019-05-01,1.030 €/m2,Murcia,Murcia,5.0,2019
77,2019-04-01,1.023 €/m2,Murcia,Murcia,4.0,2019
78,2019-03-01,1.019 €/m2,Murcia,Murcia,3.0,2019
79,2019-02-01,1.012 €/m2,Murcia,Murcia,2.0,2019


In [36]:
spain.sample(10)

,fecha,precio m2,provincia,comunidad,mes,año,trimestre
221,2020-10-01,1.175 €/m2,Cordoba,Andalucia,10.0,2020,T4
15,2024-06-01,1.245 €/m2,Murcia,Murcia,6.0,2024,T2
299,2021-01-01,1.338 €/m2,Tarragona,Cataluña,1.0,2021,T1
16,2024-05-01,2.714 €/m2,Barcelona,Cataluña,5.0,2024,T2
463,2020-11-01,850 €/m2,Jaen,Andalucia,11.0,2020,T4
5,2025-04-01,1.593 €/m2,Huesca,Aragon,4.0,2025,T2
13,2024-08-01,2.205 €/m2,Ceuta,Ceuta,8.0,2024,T3
704,2021-03-01,1.282 €/m2,Valladolid,CastillaYLeon,3.0,2021,T1
526,2022-05-01,2.589 €/m2,Malaga,Andalucia,5.0,2022,T2
395,2019-10-01,787 €/m2,Toledo,CYM,10.0,2019,T4


In [37]:
spain.columns = spain.columns.str.lower()
spain.columns = spain.columns.str.replace(' ','_')

In [ ]:
spain['precio_m2'] = spain['precio_m2'].str.replace('€/m2', '')

In [63]:
spain['precio_m2'] = spain['precio_m2'].astype(float)

In [64]:
spain.sample(2)

,fecha,precio_m2,provincia,comunidad,mes,año,trimestre
218,2021-01-01,1.172,Cordoba,Andalucia,1.0,2021,T1
167,2025-04-01,1.222,Cordoba,Andalucia,4.0,2025,T2


In [65]:
spain.to_excel('spain.xlsx')